# Horizontal flip and video in black-and-white

In [14]:
import os, random
import cv2
import numpy as np

from scipy import ndimage

video_path = '../../datasets/videos/'
output_path = '../../datasets/good_ugly_videos/'

# Get 100 random videos
# TODO: Add check for if randomly chosen video has already been processed
type = 1
for i in range(100):
  file = random.choice(os.listdir(video_path))

  vidcap = cv2.VideoCapture(video_path + file)
  fourcc = cv2.VideoWriter_fourcc(*'MJPG')
  out = cv2.VideoWriter(output_path + file, fourcc, 20.0, (int(vidcap.get(3)), int(vidcap.get(4))), 0)

  success, frame = vidcap.read()

  if type == 2:
    # Rotate with a random degree between -20 to 20
    angle = random.randint(-20, 20)
    if angle == 0:
      angle = random.randint(1, 20)

  while success:
    success, frame = vidcap.read()
    if success:
      if type == 1:
        # Add Gaussian blur with 20%
        frame = cv2.GaussianBlur(frame, (5,5), 20)

      elif type == 2:
        # Darken image to 200%
        new_frame = cv2.convertScaleAbs(frame, alpha=0.5, beta=0)
        frame = new_frame

      elif type == 3: 
        # Brighten image to 200%
        new_frame = cv2.convertScaleAbs(frame, alpha=1.5, beta=0)
        frame = new_frame

      elif type == 4:
        # Add 50% Gaussian noise
        row, col, ch = frame.shape
        mean = 0
        var = 0.1
        sigma = var ** 0.5
        noise = np.random.normal(mean, sigma, (row,col,ch)) 
        noise = noise.reshape(row,col,ch)
        noisy_frame = frame + noise
        frame = noisy_frame

      elif type == 5:
        rotated = ndimage.rotate(frame, angle)
        frame = rotated

      out.write(frame)

  # Switch to a different augmentation type
  if type == 1:
    type = 2
  elif type == 2:
    type = 3
  elif type == 3:
    type = 4
  elif type == 4:
    type = 5
  elif type ==5:
    type = 1

# Release everything if job is finished
vidcap.release()
out.release()
cv2.destroyAllWindows()